In [ ]:
def prev(ano_mes_ncm_uf, REG):
    
    import time
    ini=time.time()
    
    import pandas as pd
    
    ano_mes_ncm_uf.date=ano_mes_ncm_uf.date.astype('datetime64[ns]').astype(int)
    
    prev=pd.DataFrame()
    for date in ano_mes_ncm_uf.date.unique():
    
        ano_mes_ncm_uf_t1=ano_mes_ncm_uf[ano_mes_ncm_uf.date<=date]
    
        if len(ano_mes_ncm_uf_t1.date.unique())>=6*12:
            
            ano_mes_ncm_uf_t1=ano_mes_ncm_uf_t1.assign(oc=0)
            ano_mes_ncm_uf_t1.loc[ano_mes_ncm_uf_t1.KG_LIQUIDO!=0,'oc']=1
            
            ano_mes_ncm_uf_dumm=pd.get_dummies(ano_mes_ncm_uf_t1)
            
            train=ano_mes_ncm_uf_dumm[ano_mes_ncm_uf_dumm.date<ano_mes_ncm_uf_dumm.date.max()]
    
        
            test=ano_mes_ncm_uf_dumm[ano_mes_ncm_uf_dumm.date==ano_mes_ncm_uf_dumm.date.max()]
            test_pred=ano_mes_ncm_uf_t1[ano_mes_ncm_uf_t1.date==ano_mes_ncm_uf_t1.date.max()]
        
            X_train=train.drop(columns=['KG_LIQUIDO','VL_FOB','oc'])
            y_train_c=train.oc
            y_train_r=train.KG_LIQUIDO
            
            X_test=test.drop(columns=['KG_LIQUIDO','VL_FOB','oc'])
            y_test_c=test.oc
            y_test_r=test.KG_LIQUIDO
            
            RFC.fit(X_train,y_train_c)
    
            test_pred=test_pred.assign(oc_pred=RFC.predict(X_test))
    
            
            if (test_pred.oc.values[0]==1)&(test_pred.oc_pred.values[0]==1):
                REG.fit(X_train,y_train_r)
                test_pred=test_pred.assign(kg_pred=REG.predict(X_test))
            else:
                test_pred=test_pred.assign(kg_pred=0)
            
            prev=pd.concat([prev,test_pred])

            fim=time.time()
            
    
    return prev

    print('\nTempo Exec.:',(fim-ini)/60,' min.')

print('*** prev(ano_mes_ncm_uf, REG) function available ***')